# Import Liberaries 

In [ ]:
import json 
import re 
from sklearn.preprocessing import LabelEncoder
import numpy as np 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Read Data

In [ ]:
with open('data.json') as file:
    data=json.load(file)

In [ ]:
data

Create training example , label and response 

In [ ]:
training_example=[]
training_labels=[]
response=[]

for example in data['data']:
    for pattern in example['patterns']:
        training_example.append(pattern)
        training_labels.append(example['label'])
    response.append(example['responses'])


In [ ]:
print("training example ")
print(training_example)

In [ ]:
print("training label ")
print(training_labels)

In [ ]:
print('Response')
print(response)

In [ ]:
len(training_labels)

In [ ]:
len(set(training_labels))

Preprocessing

In [ ]:
training_example

In [ ]:
def clean_text(sentences):
    result=[]
    for sentence in sentences:
        #sentence=re.sub(r'\W',' ',sentence)# pattern  # text 
        sentence=sentence.lower()
        result.append(sentence)
        
    return result

In [ ]:
training_example=clean_text(training_example)

convert label values

In [ ]:
le=LabelEncoder()
le.fit(training_labels)
training_labels=le.fit_transform(training_labels)

In [ ]:
training_labels

In [ ]:
training_labels=to_categorical(training_labels)
training_labels

Convert words to vectors

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot 
sequnces=[one_hot(example,50,filters='') for example in training_example]
print(sequnces)

In [ ]:
toknizer=Tokenizer(num_words=100)
toknizer.fit_on_texts(training_example)
sequnces=toknizer.texts_to_sequences(training_example)
print(sequnces)

In [ ]:
padded_sequence=pad_sequences(sequnces,truncating='post',maxlen=20)
print(padded_sequence)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense ,LSTM,Embedding ,GlobalAveragePooling1D

model=Sequential()
model.add(Embedding(100,10,input_length=20))
model.add(GlobalAveragePooling1D())
model.add(Dense(10,activation='relu'))
model.add(Dense(10,activation='relu'))
model.add(Dense(8,activation='softmax'))


model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
history=model.fit(np.array(padded_sequence),training_labels,epochs=400)


Save Model

In [ ]:
inp='hello'
sent=pad_sequences(toknizer.texts_to_sequences([inp]),maxlen=20)
print(sent)
result=np.argmax(model.predict(np.array(sent))) # 0-7 
print(result)
f_res=le.inverse_transform(np.array(result).reshape(1))
print(f_res)

## Save Model


In [ ]:
import pickle as pk 
#save tokinizer
pk.dump(toknizer,open("toknizer.pkl","wb"))

In [ ]:
#save encoder
pk.dump(le,open("label_encoder.pkl","wb"))

In [ ]:
model.save('model.h5') # save model 
from tensorflow.keras.models import load_model
model=load_model('model.h5') # load model 

Interface

In [ ]:
def user_chat():
    while True :
        print("User : ",end="") 
        inp=input()

        if inp.lower()=='quit':
            break
        
        sent=pad_sequences(toknizer.texts_to_sequences([inp]),maxlen=20) #[] len=20

        result=np.argmax(model.predict(np.array(sent),verbose=0)) # 0-7 
        # 2 ,3 , 4
        f_res=le.inverse_transform(np.array(result).reshape(1))


        for label in data['data']:

            if label['label']==f_res:

                print('ChatBot : ',np.random.choice(label['responses']))



In [ ]:
print("Start Conversation with Chatbot")
user_chat()